In [1]:
import tensorflow as tf
import numpy as np

In [2]:
n_inputs = 784
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
x = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs))
y = tf.placeholder(dtype=tf.int64, shape=(None))

In [4]:
def new_layer(x, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal(shape=(n_inputs, n_neurons), stddev=stddev, dtype=tf.float32)
        w = tf.Variable(init, dtype=tf.float32)
        b = tf.Variable(tf.zeros(shape=(n_neurons)))
        z = tf.matmul(x, w) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z

In [5]:
with tf.name_scope('dnn'):
    hidden1 = new_layer(x, n_hidden1, 'hidden1', 'relu')
    hidden2 = new_layer(hidden1, n_hidden2, 'hidden2', 'relu')
    logits = new_layer(hidden2, n_outputs, 'outputs')

In [6]:
with tf.name_scope('loss'):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(entropy)

In [7]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [8]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    acuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/home/anand/.tensorflow/data/MNIST")

Extracting /home/anand/.tensorflow/data/MNIST/train-images-idx3-ubyte.gz
Extracting /home/anand/.tensorflow/data/MNIST/train-labels-idx1-ubyte.gz
Extracting /home/anand/.tensorflow/data/MNIST/t10k-images-idx3-ubyte.gz
Extracting /home/anand/.tensorflow/data/MNIST/t10k-labels-idx1-ubyte.gz


In [ ]:
n_epocs = 50
batch_size = 50

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epocs):
        for iteration in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
        acc_train = acuracy.eval(feed_dict={x: x_batch, y: y_batch})
        acc_test = acuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})
        
        print(epoch, "Train accuracy", acc_train, "Test accuracy", acc_test)
        
    save_path = saver.save(sess, "/tmp/model/mnist_final.ckpt")

In [20]:
print(mnist.test.images[51].shape)


(784,)
3


In [24]:
idx = 7

with tf.Session() as sess:
    save_path = saver.restore(sess, "/tmp/model/mnist_final.ckpt")
    
    image = mnist.test.images[idx].reshape(-1, 784)
    l = sess.run(logits, feed_dict={x: image})
    print("Prediction", np.argmax(l, axis=1))
    print("Actual", mnist.test.labels[idx])

INFO:tensorflow:Restoring parameters from /tmp/model/mnist_final.ckpt
Prediction [9]
Actual 9
